In [9]:
from pymongo import MongoClient
import time
import operator
import datetime
import re

client = MongoClient()
db = client.twitter
te = db['prueba']
dia_inicio = datetime.datetime.now()
dia_inicio = datetime.datetime(dia_inicio.year, dia_inicio.month, dia_inicio.day -1)
print dia_inicio

2017-01-22 00:00:00


In [10]:
orig = te.find_one({'created_at': {"$gt" : dia_inicio}}, sort=[("created_at", 1)])["created_at"]
fin = te.find_one({'created_at': {"$gt" : dia_inicio}}, sort=[("created_at", -1)])["created_at"]
intervalo = 15
l= []
#trunco el a a minutos
a =  datetime.datetime(orig.year, orig.month, orig.day, orig.hour, orig.minute)
while a < fin: 
    sl = []
    b = a + datetime.timedelta(minutes=intervalo)
    rl = te.find({'$and': [{'created_at': {"$gte" : a}} ,{'created_at': {"$lt" : b}} ]})
    r = rl.count()
    if r >0:
        #rh = te.find({'$and': [{'text':{"$regex" : '#'}} ,{'created_at': {"$gte" : a}} ,{'created_at': {"$lt" : b}} ]}).count()
        #rhl = te.find({'$and': [{'text':{"$regex" : '#'}} ,{'created_at': {"$gte" : a}} ,{'created_at': {"$lt" : b}} ]})
        #rw = te.find({'$and': [{'text':{"$regex" : 'http'}} ,{'created_at': {"$gte" : a}} ,{'created_at': {"$lt" : b}} ]}).count()
        #rh2 = [tw for tw in r if tw['entities']['hastags']]
        #rw2 = [tw for tw in r if len(tw['entities']['urls'])>0   ]
        rw = len(filter(lambda a: len(a['entities']['urls']) >0, rl))
        rl.rewind()
        rh = len(filter(lambda a: len(a['entities']['hashtags']) >0, rl))
        sl.append(a)
        sl.append(r)
        sl.append(rh)
        sl.append(rw)
        l.append(sl)
    a=b 


In [15]:
ls = sorted(l, key=operator.itemgetter(0))
for a in ls: 
    print str(a[0]) + ' ====> ' + str(a[1]) + ' ' + "{0:.2f}".format((a[2]/float(a[1]) * 100)) + '% ' + "{0:.2f}".format((a[3]/float(a[1]) * 100)) + '%'
    
    str(a[2]/float(a[1]) * 100)

2017-01-23 18:18:00 ====> 34 11.76% 94.12%


In [5]:
## get url Real-- no lo usamos, esta en entities
import urllib2

class HeadRequest(urllib2.Request):
    def get_method(self): return "HEAD"
    
def get_real(url):
    res = urllib2.urlopen(HeadRequest(url))
    return res.geturl()

In [6]:
###Cogemos las urls de un intervalo
def get_urls(init, size):
    d = {}
    end = init + datetime.timedelta(minutes=size)
    rw = te.find({'$and': [{'text':{"$regex" : 'http'}} ,{'created_at': {"$gte" : init}} ,{'created_at': {"$lt" : end}} ]})
    for t in rw: 
        #web = re.search("(?P<url>https?://[^\s]+)", t['text']).group("url") 
        #print get_real(web)
        for w in t['entities']['urls']:
            if w['expanded_url'] in d:
                d[w['expanded_url']] = d[w['expanded_url']] + 1
            else:
                d[w['expanded_url']] =1
    
    return d
init = datetime.datetime(2017, 1, 22, 0, 0)
u = get_urls(init, 3600)

In [7]:
sorted(u.items(), key=operator.itemgetter(1), reverse=True)
    


[(u'https://www.youtube.com/watch?v=qmujL4izDrU', 553),
 (u'https://www.youtube.com/watch?v=0Dgvp7C2wlI', 553),
 (u'https://www.youtube.com/watch?v=NE3435z5p-Y&t=5s', 553),
 (u'http://www.trendinalia.com/twitter-trending-topics/spain/spain-today.html',
  63),
 (u'http://xornaldegalicia.es/especiales/40-farandula/9614-medicos-y-abogados-enjuiciados-moralmente-en-un-escenario-conflictos-de-pasiones-y-resentimientos-miedos-y-ambiciones-en-casos-de-negligencias',
  58),
 (u'https://twitter.com/A3Noticias/status/823120023506079744', 45),
 (u'https://twitter.com/SweetOfCharlie/status/823179676210315267', 42),
 (u'http://www.vanitatis.elconfidencial.com/noticias/2015-07-23/los-800-000-euros-que-hacienda-le-reclama-a-maria-teresa-campos_940247/',
  38),
 (u'http://www.lavanguardia.com/local/sevilla/20160317/40495431317/novio-paz-padill-imputado-juan-vidal.html',
  37),
 (u'http://www.estrelladigital.es/blog/mosca/belen-esteban-ejerce-vito-corleone/20161107141521303495.html',
  36),
 (u'http://

In [8]:
###Creamos rangos y cogemos lo más visitado
d = {}
for a in te.find({'$and': [{'text':{"$regex" : '#'}} ,{'created_at': {"$gt" : dia_inicio}} ]},  { 'text': 1}):
	s = a['text']
	for ht in  [b.strip('#') for b in s.split() if b.startswith("#")]:
		if ht not in d:
			d[ht] = 1
		else:
			d[ht] = d[ht] + 1
trending = sorted(d.items(), key=operator.itemgetter(1))
print trending


[(u'instarelax', 1), (u'yellow', 1), (u'Olympics', 1), (u'SanzyTorres', 1), (u'sanantonio', 1), (u'hermana', 1), (u'EsparTo\xf1osypnvep', 1), (u'roadtohome', 1), (u'neu\u2026', 1), (u'LevantevsNumancia', 1), (u'BiosCidob', 1), (u'barriohumedo', 1), (u'florculo', 1), (u'falles\u2026', 1), (u'openwash', 1), (u'enlaVeterinaria', 1), (u'museoprincipefelipe', 1), (u'modernasjaja\u2026', 1), (u'fonTETA\u2026', 1), (u'Jerez', 1), (u'Godoff', 1), (u'FranMinaya', 1), (u'mesientogenial', 1), (u'Golsolidario', 1), (u'sombreado\u2026', 1), (u'CulleradeFusta2017!', 1), (u'pluja', 1), (u'ViolenceAgainstWomen\u2026', 1), (u'brexit', 1), (u'CursaSantAntoni,', 1), (u'MarqAlicante', 1), (u'feellikeaninja', 1), (u'VamosUCAM\U0001f4aa', 1), (u'fitur2017,', 1), (u'preventivoscruzroja', 1), (u'vocacionales.', 1), (u'personalizedlearning?', 1), (u'culo', 1), (u'CastroCandaz', 1), (u'bedifferent', 1), (u'Llanes', 1), (u'AusOpen,despejado', 1), (u'MilitantesEnPie!!', 1), (u'joven\u2026', 1), (u'china', 1), (u'